In [38]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, year, month, monotonically_increasing_id


spark = SparkSession.builder \
    .appName("ETL to Star Schema") \
    .config("spark.jars", "/home/jovyan/work/postgresql-42.6.0.jar") \
    .getOrCreate()

pg_url = "jdbc:postgresql://bigdata_postgres_db:5432/lab2"
pg_props = {
    "user": "debug",
    "password": "pswd",
    "driver": "org.postgresql.Driver"
}


In [2]:
df = spark.read.jdbc(pg_url, "mock_data", properties=pg_props)
df.printSchema()
df.show(1)

root
 |-- id: integer (nullable = true)
 |-- customer_first_name: string (nullable = true)
 |-- customer_last_name: string (nullable = true)
 |-- customer_age: integer (nullable = true)
 |-- customer_email: string (nullable = true)
 |-- customer_country: string (nullable = true)
 |-- customer_postal_code: string (nullable = true)
 |-- customer_pet_type: string (nullable = true)
 |-- customer_pet_name: string (nullable = true)
 |-- customer_pet_breed: string (nullable = true)
 |-- seller_first_name: string (nullable = true)
 |-- seller_last_name: string (nullable = true)
 |-- seller_email: string (nullable = true)
 |-- seller_country: string (nullable = true)
 |-- seller_postal_code: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- product_price: decimal(10,2) (nullable = true)
 |-- product_quantity: integer (nullable = true)
 |-- sale_date: timestamp (nullable = true)
 |-- sale_customer_id: integer (nullable = tru

In [9]:
from pyspark.sql.functions import col

customers_df = df.select(
    col("customer_first_name").alias("first_name"),
    col("customer_last_name").alias("last_name"),
    col("customer_age").alias("age"),
    col("customer_email").alias("email"),
    col("customer_country").alias("country"),
    col("customer_postal_code").alias("postal_code"),
    col("customer_pet_type").alias("pet_type"), 
    col("customer_pet_name").alias("pet_name"), 
    col("customer_pet_breed").alias("pet_breed"), 
    "pet_category"
).dropDuplicates()

# загружаем в postgres в таблицу d_customers
customers_df.write.jdbc(url=pg_url, table="d_customers", mode="append", properties=pg_props)


In [10]:
from pyspark.sql.functions import col

products_df = df.select(
    col("product_name").alias("name"),
    col("product_category").alias("category"),
    col("product_price").alias("price"),
    col("product_quantity").alias("quantity"),
    col("product_weight").alias("weight"),
    col("product_color").alias("color"),
    col("product_size").alias("size"),
    col("product_brand").alias("brand"),
    col("product_material").alias("material"),
    col("product_description").alias("description"),
    col("product_rating").alias("rating"),
    col("product_reviews").alias("reviews"),
    col("product_release_date").alias("release_date"),
    col("product_expiry_date").alias("expiry_date")
).dropDuplicates()

# загружает в postgres в таблицу d_products
products_df.write.jdbc(url=pg_url, table="d_products", mode="append", properties=pg_props)

In [11]:
from pyspark.sql.functions import col

sellers_df = df.select(
    col("seller_first_name").alias("first_name"),
    col("seller_last_name").alias("last_name"),
    col("seller_email").alias("email"),
    col("seller_country").alias("country"),
    col("seller_postal_code").alias("postal_code")
).dropDuplicates()

# загружает в postgres в таблицу d_sellers
sellers_df.write.jdbc(url=pg_url, table="d_sellers", mode="append", properties=pg_props)

In [12]:
from pyspark.sql.functions import col

suppliers_df = df.select(
    col("supplier_name").alias("name"),
    col("supplier_contact").alias("contact"),
    col("supplier_email").alias("email"),
    col("supplier_phone").alias("phone"),
    col("supplier_address").alias("address"),
    col("supplier_city").alias("city"),
    col("supplier_country").alias("country")
).dropDuplicates()

# загружает в postgres в таблицу d_suppliers
suppliers_df.write.jdbc(url=pg_url, table="d_suppliers", mode="append", properties=pg_props)

In [13]:
from pyspark.sql.functions import col

stores_df = df.select(
    col("store_name").alias("name"),
    col("store_location").alias("location"),
    col("store_city").alias("city"),
    col("store_state").alias("state"),
    col("store_country").alias("country"),
    col("store_phone").alias("phone"),
    col("store_email").alias("email")
).dropDuplicates()

# загружает в postgres в таблицу d_stores
stores_df.write.jdbc(url=pg_url, table="d_stores", mode="append", properties=pg_props)

In [14]:
from pyspark.sql.functions import col, year, month, dayofmonth, dayofweek

times_df = df.select(
    col("sale_date").alias("date")
).dropDuplicates()

times_df = times_df.withColumn("year", year("date")) \
                   .withColumn("month", month("date")) \
                   .withColumn("day", dayofmonth("date")) \
                   .withColumn("weekday", dayofweek("date"))

# загружает в postgres в таблицу d_times
times_df.write.jdbc(url=pg_url, table="d_times", mode="append", properties=pg_props)

Теперь свяжем все данные с таблицей фактов.

In [19]:
# берем данные мз таблиц измерений
dim_customers = spark.read.jdbc(url=pg_url, table="d_customers", properties=pg_props)
dim_sellers = spark.read.jdbc(url=pg_url, table="d_sellers", properties=pg_props)
dim_products = spark.read.jdbc(url=pg_url, table="d_products", properties=pg_props)
dim_stores = spark.read.jdbc(url=pg_url, table="d_stores", properties=pg_props)
dim_suppliers = spark.read.jdbc(url=pg_url, table="d_suppliers", properties=pg_props)
dim_times = spark.read.jdbc(url=pg_url, table="d_times", properties=pg_props)

Теперь соединим данные в соответсвии с исходной таблицей.

In [27]:
from pyspark.sql.functions import col, broadcast, monotonically_increasing_id

# alias для читаемости
raw = df.alias("raw")
cust = dim_customers.alias("cust")
sell = dim_sellers.alias("sell")
prod = dim_products.alias("prod")
store = dim_stores.alias("store")
sup = dim_suppliers.alias("sup")
dt = dim_times.alias("dt")

# JOIN всех измерений
fact_df = (
    raw
    .join(cust,
          (raw["customer_email"] == cust["email"]),
          "left")
    .join(sell,
          (raw["seller_email"] == sell["email"]),
          "left")
    .join(prod,
          (raw["product_name"] == prod["name"]) &
          (raw["product_category"] == prod["category"]) &
          (raw["product_price"] == prod["price"]) &
          (raw["product_brand"] == prod["brand"]) &
          (raw["product_size"] == prod["size"]) &
          (raw["product_material"] == prod["material"]),
          "left")
    .join(store,
          (raw["store_email"] == store["email"]),
          "left")
    .join(sup,
          (raw["supplier_email"] == sup["email"]),
          "left")
    .join(broadcast(dt), raw["sale_date"] == dt["date"], "left")
    .select(
        col("dt.id").alias("date_id"),
        col("cust.id").alias("customer_id"),
        col("sell.id").alias("seller_id"),
        col("prod.id").alias("product_id"),
        col("store.id").alias("store_id"),
        col("sup.id").alias("supplier_id"),
        col("raw.sale_quantity").cast("int").alias("quantity"),
        col("raw.sale_total_price").cast("decimal(10,2)").alias("total_price")
    )
)
print(fact_df.count())

10000


На этапе join возникли проблемы, так как делал слияния с ошибкой --- не уникальные поля брал, в следствие чего получил ~11000000 записей в `f_sales` и долгую запись.

Отбираем только нужные столбцы.

In [28]:
fact_df = fact_df.repartition(4)

# Запись с настройками
fact_df.write \
    .format("jdbc") \
    .option("url", pg_url) \
    .option("dbtable", "f_sales") \
    .option("user", pg_props["user"]) \
    .option("password", pg_props["password"]) \
    .option("driver", "org.postgresql.Driver") \
    .option("batchsize", "500") \
    .option("truncate", "true") \
    .mode("overwrite") \
    .save()


В данном случае нет необходимости использовать батчевую загрузку, 10000 не так много.

## Работа с clickhouse

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark report to clickhouse") \
    .config("spark.jars", "clickhouse-jdbc-0.4.6.jar,postgresql-42.6.0.jar") \
    .getOrCreate()

ch_url = "jdbc:clickhouse://clickhouse:8123/default"
ch_props = {
    "user": "custom_user",
    "password": "pswd",
    "driver": "com.clickhouse.jdbc.ClickHouseDriver"
}

pg_url = "jdbc:postgresql://bigdata_postgres_db:5432/lab2"
pg_props = {
    "user": "debug",
    "password": "pswd",
    "driver": "org.postgresql.Driver"
}


### Отчет по продажам

In [11]:
fact_df = spark.read.jdbc(url=pg_url, table="f_sales", properties=pg_props)

dm_products = spark.read.jdbc(url=pg_url, table="d_products", properties=pg_props)

Достаем топ-10 самых продаваемых продуктов.

In [12]:
from pyspark.sql.functions import col, sum as _sum

top_products = (
    fact_df
    .join(dm_products, fact_df.product_id == dm_products.id)
    .groupBy(dm_products["name"])
    .agg(_sum(fact_df["quantity"]).alias("total_quantity"))
    .orderBy(col("total_quantity").desc())
    .limit(10)
)

top_products.show()

+---------+--------------+
|     name|total_quantity|
+---------+--------------+
| Dog Food|         18298|
|Bird Cage|         18205|
|  Cat Toy|         18120|
+---------+--------------+



Общая выручка по категориям продуктов.

In [13]:
from pyspark.sql.functions import sum as _sum, col

revenue_by_category = (
    fact_df
    .join(dm_products, fact_df.product_id == dm_products.id)
    .groupBy(dm_products["category"])
    .agg(_sum(fact_df["total_price"]).alias("total_revenue"))
    .orderBy(col("total_revenue").desc())
)

revenue_by_category.show()

+--------+-------------+
|category|total_revenue|
+--------+-------------+
|     Toy|    868101.63|
|    Cage|    831117.94|
|    Food|    830632.55|
+--------+-------------+



Средний рейтинг и количество отзывов для каждого продукта.

In [29]:
from pyspark.sql.functions import avg, max, sum

ratings_reviews = (
    dm_products
    .groupBy("name")
    .agg(
        avg("rating").alias("avg_rating"),
        sum("reviews").alias("total_reviews")
    )
)
ratings_reviews.show()

+---------+----------+-------------+
|     name|avg_rating|total_reviews|
+---------+----------+-------------+
|Bird Cage| 3.0001492|      1682260|
| Dog Food| 3.0182989|      1653413|
|  Cat Toy| 3.0068601|      1676222|
+---------+----------+-------------+



In [62]:
from pyspark.sql.functions import lit, to_json, struct

df_top10_json = top_products \
    .withColumn("report_type", lit("top_10_products")) \
    .withColumn("report_data", to_json(struct(*top_products.columns))) \
    .select("report_type", "report_data")

df_revenue_json = revenue_by_category \
    .withColumn("report_type", lit("revenue_by_category")) \
    .withColumn("report_data", to_json(struct(*revenue_by_category.columns))) \
    .select("report_type", "report_data")

df_ratings_json = ratings_reviews \
    .withColumn("report_type", lit("ratings_reviews")) \
    .withColumn("report_data", to_json(struct(*ratings_reviews.columns))) \
    .select("report_type", "report_data")

df_reports = df_top10_json.union(df_revenue_json).union(df_ratings_json)

In [63]:
df_reports.write.mode("append").jdbc(url=ch_url, table="report_product", properties=ch_props)

### Отчёт по клиентам

In [27]:
f_sales = spark.read.jdbc(url=pg_url, table="f_sales", properties=pg_props)
d_customers = spark.read.jdbc(url=pg_url, table="d_customers", properties=pg_props)

Топ-10 клиентов с наибольшей общей суммой покупок.

In [28]:
from pyspark.sql.functions import sum as _sum

top_customers = (
    f_sales.join(d_customers, f_sales.customer_id == d_customers.id)
    .groupBy("customer_id", "first_name", "last_name", "country")
    .agg(_sum("total_price").alias("total_spent"))
    .orderBy("total_spent", ascending=False)
    .limit(10)
)

top_customers.show()

+-----------+----------+----------+---------+-----------+
|customer_id|first_name| last_name|  country|total_spent|
+-----------+----------+----------+---------+-----------+
|       5408|       Gus| Hartshorn|  Albania|     499.85|
|       1770|     Hayes|    McKain| Portugal|     499.80|
|       4593|       Ava|     Lomas|    China|     499.76|
|       4423|     Dawna|     Impey|Indonesia|     499.76|
|       3357|   Lavinia| Horsburgh|   Poland|     499.73|
|       8018|      Dame|Auchinleck|Indonesia|     499.71|
|       1048|  Isahella|    Colley|   Russia|     499.69|
|       5937|     Nicky|    Lattie|   Mexico|     499.62|
|       3025|    Sisely|  Bonevant|    China|     499.62|
|       5130|      Eran|     Cotes|    China|     499.59|
+-----------+----------+----------+---------+-----------+



Распределение клиентов по странам.

In [40]:
from pyspark.sql.functions import count

country_distribution = (
    d_customers.groupBy("country").agg(count("*").alias("count_of_customers"))
)

country_distribution.show()

+--------------------+------------------+
|             country|count_of_customers|
+--------------------+------------------+
|                Chad|                 5|
|              Russia|               628|
|            Paraguay|                18|
|               Yemen|                39|
| U.S. Virgin Islands|                 1|
|             Senegal|                 4|
|              Sweden|               264|
|Svalbard and Jan ...|                 1|
|              Guyana|                 1|
|         Philippines|               555|
|             Eritrea|                 3|
|            Djibouti|                 1|
|            Malaysia|                40|
|              Turkey|                 1|
|              Malawi|                12|
|                Iraq|                 8|
|             Germany|                30|
|Northern Mariana ...|                 1|
|             Comoros|                13|
|         Afghanistan|                31|
+--------------------+------------

Средний чек для каждого клиента.

In [51]:
from pyspark.sql.functions import count, sum, round

avg_check = (
    f_sales.join(d_customers, f_sales.customer_id == d_customers.id).groupBy("customer_id", "first_name", "last_name")
    .agg(round(_sum("total_price") / count(f_sales.id), 2).alias("avg_check"))
)

avg_check.show()

+-----------+----------+----------+---------+
|customer_id|first_name| last_name|avg_check|
+-----------+----------+----------+---------+
|        148|    Sander|  Warhurst|   308.63|
|        463|       Dov|   Stanton|   372.40|
|        471|     Hedda|   Enrrico|   334.62|
|        496|   Sheeree|  Matthias|   383.82|
|        833|   Merrick| Shotboult|   377.44|
|       1088|      Cori|    Spragg|   463.65|
|       1238|      Rois|   Byfford|    65.36|
|       1342|      Alla|      Jore|   365.57|
|       1580|     Tarah|   Scanlin|    14.63|
|       1591|    Jasper|   Antonat|   378.88|
|       1645|  Tomasina|     Bound|   163.33|
|       1829|       Eve|   Cheshir|   480.87|
|       1959|   Donovan|     Toupe|   302.61|
|       2122|   Camilla|    Kieran|    83.58|
|       2142|   Corenda|    Allatt|    28.17|
|       2366|    Livvie|  Mountjoy|    64.11|
|       2659|   Carilyn|      Glyn|    71.86|
|       2866|    Roarke|     Kobus|    16.43|
|       3175|    Tracey|McAllaster

Объединяем фреймы в один

In [61]:
from pyspark.sql.functions import lit, to_json, struct

df_top_customers_json = top_customers \
    .withColumn("report_type", lit("top_10_customers")) \
    .withColumn("report_data", to_json(struct(*top_customers.columns))) \
    .select("report_type", "report_data")

df_distribution_json = country_distribution \
    .withColumn("report_type", lit("distribution_by_country")) \
    .withColumn("report_data", to_json(struct(*country_distribution.columns))) \
    .select("report_type", "report_data")

df_avg_check_json = avg_check \
    .withColumn("report_type", lit("avg_check")) \
    .withColumn("report_data", to_json(struct(*avg_check.columns))) \
    .select("report_type", "report_data")

df_reports = df_top_customers_json.union(df_distribution_json).union(df_avg_check_json)

In [60]:
df_reports.write.mode("append").jdbc(url=ch_url, table="report_customer", properties=ch_props)

###  Отчёт. Витрина продаж по времени

In [70]:
f_sales = spark.read.jdbc(url=pg_url, table="f_sales", properties=pg_props)
d_times = spark.read.jdbc(url=pg_url, table="d_times", properties=pg_props)

Месячные и годовые тренды продаж.

In [90]:
from pyspark.sql.window import Window
from pyspark.sql.functions import sum as _sum

# обычная агрегация по году и месяцу
monthly_yearly_trends = (
    f_sales.join(d_times, f_sales.date_id == d_times.id)
    .groupBy("year", "month")
    .agg(_sum("total_price").alias("total_revenue"))
)

# окно по каждому году
year_window = Window.partitionBy("year")

# добавим столбец с годовой выручкой
monthly_with_year_total = monthly_yearly_trends.withColumn(
    "year_total_revenue",
    _sum("total_revenue").over(year_window)
).orderBy("year", "month")

monthly_with_year_total.show()

+----+-----+-------------+------------------+
|year|month|total_revenue|year_total_revenue|
+----+-----+-------------+------------------+
|2021|    1|    224158.54|        2529852.12|
|2021|    2|    192348.31|        2529852.12|
|2021|    3|    207282.20|        2529852.12|
|2021|    4|    206592.82|        2529852.12|
|2021|    5|    211764.86|        2529852.12|
|2021|    6|    215042.80|        2529852.12|
|2021|    7|    220496.51|        2529852.12|
|2021|    8|    221275.78|        2529852.12|
|2021|    9|    210623.43|        2529852.12|
|2021|   10|    228743.32|        2529852.12|
|2021|   11|    200154.69|        2529852.12|
|2021|   12|    191368.86|        2529852.12|
+----+-----+-------------+------------------+



Сравнение выручки за разные периоды.

In [79]:
from pyspark.sql.functions import sum as _sum, count, round, when


d_times_with_season = d_times.withColumn(
    "season",
    when((d_times["month"].isin(12, 1, 2)), "winter")
    .when((d_times["month"].isin(3, 4, 5)), "spring")
    .when((d_times["month"].isin(6, 7, 8)), "summer")
    .when((d_times["month"].isin(9, 10, 11)), "autumn")
)

avg_revenue_by_season = (
    f_sales.join(d_times_with_season, f_sales.date_id == d_times_with_season.id)
    .groupBy("season")
    .agg(
        _sum("total_price").alias("total_revenue"),
        count(f_sales.id).alias("orders_count"),
        round(_sum("total_price") / count(f_sales.id), 2).alias("avg_revenue")
    )

)

avg_revenue_by_season.show()

+------+-------------+------------+-----------+
|season|total_revenue|orders_count|avg_revenue|
+------+-------------+------------+-----------+
|winter|    607875.71|        2383|     255.09|
|summer|    656815.09|        2577|     254.88|
|spring|    625639.88|        2508|     249.46|
|autumn|    639521.44|        2532|     252.58|
+------+-------------+------------+-----------+



Средний размер заказа по месяцам.

*Так как у нас данные за один год 2021, то не вывожу его.*

In [86]:
from pyspark.sql.functions import sum as _sum, count, round, col

avg_order_by_month = (
    f_sales.join(d_times, f_sales.date_id == d_times.id)
    .groupBy("month")
    .agg(
        _sum("total_price").alias("total_revenue"),
        count(f_sales.id).alias("orders_count"),
        round(_sum("total_price") / count(f_sales.id), 2).alias("avg_order_size")
    )
    .orderBy("month")
)

avg_order_by_month.show()

+-----+-------------+------------+--------------+
|month|total_revenue|orders_count|avg_order_size|
+-----+-------------+------------+--------------+
|    1|    224158.54|         874|        256.47|
|    2|    192348.31|         739|        260.28|
|    3|    207282.20|         843|        245.89|
|    4|    206592.82|         837|        246.83|
|    5|    211764.86|         828|        255.75|
|    6|    215042.80|         822|        261.61|
|    7|    220496.51|         858|        256.99|
|    8|    221275.78|         897|        246.68|
|    9|    210623.43|         839|        251.04|
|   10|    228743.32|         892|        256.44|
|   11|    200154.69|         801|        249.88|
|   12|    191368.86|         770|        248.53|
+-----+-------------+------------+--------------+



Собираем аналитику вместе и сохраняем.

In [93]:
from pyspark.sql.functions import lit, to_json, struct


df_revenue_json = monthly_with_year_total \
    .withColumn("report_type", lit("revenue_by_month_and_year")) \
    .withColumn("report_data", to_json(struct(*monthly_with_year_total.columns))) \
    .select("report_type", "report_data")

df_avg_order_json = avg_order_by_month \
    .withColumn("report_type", lit("avg_order_by_month")) \
    .withColumn("report_data", to_json(struct(*avg_order_by_month.columns))) \
    .select("report_type", "report_data")

df_avg_season_json = avg_revenue_by_season \
    .withColumn("report_type", lit("avg_revenue_by_season")) \
    .withColumn("report_data", to_json(struct(*avg_revenue_by_season.columns))) \
    .select("report_type", "report_data")

df_reports = df_revenue_json.union(df_avg_order_json).union(df_avg_season_json)


In [94]:
df_reports.write.mode("append").jdbc(url=ch_url, table="report_time", properties=ch_props)

### Витрина продаж по магазинам

In [2]:
f_sales = spark.read.jdbc(url=pg_url, table="f_sales", properties=pg_props)
d_stores = spark.read.jdbc(url=pg_url, table="d_stores", properties=pg_props)

Топ-5 магазинов с наибольшей выручкой.

In [13]:
from pyspark.sql.functions import sum

top_5_stores = (
    f_sales.groupBy("store_id")
    .agg(sum("total_price").alias("revenue"))
    .join(d_stores, f_sales.store_id == d_stores.id)
    .select("name", "city", "country", "revenue")
    .orderBy("revenue", ascending=False)
    .limit(5)
)

top_5_stores.show()

+-----------+---------+------------+-------+
|       name|     city|     country|revenue|
+-----------+---------+------------+-------+
|       DabZ|   Grekan|South Africa| 499.85|
|Thoughtblab|    Fonte|      Poland| 499.80|
|   Edgeblab|    Pesek|   Indonesia| 499.76|
|     Camido|Longzhong|      Sweden| 499.76|
|    Centizu|   Tylicz|      Poland| 499.73|
+-----------+---------+------------+-------+



Распределение продаж по городам и странам.

In [9]:
from pyspark.sql.functions import sum

sales_by_location = (
    f_sales.join(d_stores, f_sales.store_id == d_stores.id)
    .groupBy("country", "city")
    .agg(sum("total_price").alias("total_revenue"),
        sum("quantity").alias("total_quantity"))
    .orderBy("country", "city")
)

sales_by_location.show()

+-----------+-------------+-------------+--------------+
|    country|         city|total_revenue|total_quantity|
+-----------+-------------+-------------+--------------+
|Afghanistan|     Alingsås|       166.11|             4|
|Afghanistan|    Asheville|       466.53|            10|
|Afghanistan|        Asker|       375.17|             2|
|Afghanistan|   Berkovitsa|       167.99|            10|
|Afghanistan|     Borūjerd|       492.22|            10|
|Afghanistan|Calzada Larga|       457.36|             3|
|Afghanistan| Cruz del Eje|       187.23|             5|
|Afghanistan|       Daugai|       485.66|             5|
|Afghanistan|     Ddmashen|        65.99|             2|
|Afghanistan|   El Rosario|       359.37|             6|
|Afghanistan|      Gagarin|       301.95|             9|
|Afghanistan|      Gjinkar|       387.76|            10|
|Afghanistan|Golema Rečica|       385.67|             4|
|Afghanistan| Grand Rapids|       241.88|             8|
|Afghanistan|      Guankou|    

Средний чек для каждого магазина.

In [22]:
from pyspark.sql.functions import sum as _sum, count, round, col

avg_check_by_store = (
    f_sales.join(d_stores, f_sales.store_id == d_stores.id)
    .groupBy(d_stores.id, "name")
    .agg(
        _sum("total_price").alias("total_revenue"),
        count("*").alias("orders_count"),
    )
    .withColumn("avg_check", round(col("total_revenue") / col("orders_count"), 2))
    .select("name", "avg_check")
)

avg_check_by_store.show()

+-------------+---------+
|         name|avg_check|
+-------------+---------+
|         Mymm|   360.04|
|         Jayo|   172.76|
|     Livetube|    46.63|
|       Skimia|   174.15|
|      Voolith|    14.11|
|        Vimbo|    39.42|
| Jabbersphere|   195.98|
|      Dabvine|   308.35|
|     Gigazoom|   399.54|
|       Voolia|   238.61|
|   Topicshots|   443.13|
|      Pixonyx|   325.13|
|        Aimbu|   237.52|
|      Dabtype|   101.05|
|      Gabcube|   499.21|
|        Eadel|    32.03|
|Twitternation|   273.09|
|    Bubblebox|   164.21|
|         Lajo|   478.60|
|  Shufflester|   446.64|
+-------------+---------+
only showing top 20 rows



In [23]:
from pyspark.sql.functions import lit, to_json, struct

df_top5_json = top_5_stores \
    .withColumn("report_type", lit("top_5_stores")) \
    .withColumn("report_data", to_json(struct(*top_5_stores.columns))) \
    .select("report_type", "report_data")

df_sales_location_json = sales_by_location \
    .withColumn("report_type", lit("sales_by_location")) \
    .withColumn("report_data", to_json(struct(*sales_by_location.columns))) \
    .select("report_type", "report_data")

df_avg_check_json = avg_check_by_store \
    .withColumn("report_type", lit("avg_check_by_store")) \
    .withColumn("report_data", to_json(struct(*avg_check_by_store.columns))) \
    .select("report_type", "report_data")

df_store_reports = df_top5_json.union(df_sales_location_json).union(df_avg_check_json)

In [24]:
df_store_reports.write.mode("append").jdbc(url=ch_url, table="store_report", properties=ch_props)

### Продажи по поставщикам

In [25]:
f_sales = spark.read.jdbc(url=pg_url, table="f_sales", properties=pg_props)
d_suppliers = spark.read.jdbc(url=pg_url, table="d_suppliers", properties=pg_props)

Топ-5 поставщиков с наибольшей выручкой.

In [30]:
from pyspark.sql.functions import sum

top_5_suppliers = (
    f_sales.groupBy("supplier_id")
    .agg(sum("total_price").alias("revenue"))
    .join(d_suppliers, f_sales.supplier_id == d_suppliers.id)
    .select("name", "city", "country", "revenue")
    .orderBy("revenue", ascending=False)
    .limit(5)
)

top_5_suppliers.show()

+----------+----------+---------+-------+
|      name|      city|  country|revenue|
+----------+----------+---------+-------+
|Brainverse|    Tudela|  Ireland| 499.85|
|     Jamia|     Luleå|   Russia| 499.80|
|     Eabox|   Sloboda| Portugal| 499.76|
|   Demimbu|   Begejci|    China| 499.76|
|Browsezoom|San Isidro|Argentina| 499.73|
+----------+----------+---------+-------+



Средняя цена товаров от каждого поставщика.

In [32]:
d_products = spark.read.jdbc(url=pg_url, table="d_products", properties=pg_props)

In [36]:
from pyspark.sql.functions import avg

avg_price_by_supplier = (
    f_sales.join(d_products, f_sales.product_id == d_products.id)
           .join(d_suppliers, f_sales.supplier_id == d_suppliers.id)
           .groupBy("supplier_id", d_suppliers.name, "country")
           .agg(avg("price").alias("avg_product_price"))
)

avg_price_by_supplier.show()

+-----------+-----------+-----------+-----------------+
|supplier_id|       name|    country|avg_product_price|
+-----------+-----------+-----------+-----------------+
|       5429|   Skipfire|Philippines|        87.030000|
|       3008|      Mycat|     Brazil|        50.420000|
|       2114|    Youfeed|  Indonesia|        39.160000|
|       8231|      Jatri|      Tonga|        24.760000|
|       8042|      Aivee|     Brazil|        93.890000|
|       1105|     Avavee|      China|        78.290000|
|       4738|      Plajo|      China|        48.230000|
|       8849|    Youspan|     Poland|        90.620000|
|       9248|  Flipstorm|   Portugal|        15.720000|
|       3451|      Mynte|  Indonesia|        67.920000|
|       1545| Thoughtmix|    Nigeria|        99.270000|
|       3025|Jabberstorm|      China|        19.440000|
|       4688|    Tagfeed|  Indonesia|         9.170000|
|       6584|      Eabox|  Macedonia|        37.480000|
|       4886|   Gigaclub|  Indonesia|        11.

Распределение продаж по странам поставщиков.

In [38]:
from pyspark.sql.functions import sum

sales_by_supplier_country = (
    f_sales.join(d_suppliers, f_sales.supplier_id == d_suppliers.id)
    .groupBy("country")
    .agg(sum("total_price").alias("revenue"))
    .orderBy("revenue", ascending=False)
)

sales_by_supplier_country.show()

+--------------+---------+
|       country|  revenue|
+--------------+---------+
|         China|492823.31|
|     Indonesia|265717.99|
|        Russia|149206.75|
|   Philippines|136135.10|
|        Brazil| 97546.82|
|        Poland| 87370.64|
|      Portugal| 83210.60|
|        France| 80432.46|
| United States| 52560.14|
|        Sweden| 52074.94|
|Czech Republic| 45258.81|
|       Ukraine| 42858.17|
|      Thailand| 42409.05|
|         Japan| 42075.85|
|      Colombia| 39525.61|
|          Peru| 37351.86|
|     Argentina| 35606.32|
|        Greece| 33146.62|
|        Canada| 30290.58|
|       Nigeria| 24725.21|
+--------------+---------+
only showing top 20 rows



In [41]:
from pyspark.sql.functions import lit, to_json, struct

df_top5_json = top_5_suppliers \
    .withColumn("report_type", lit("top_5_suppliers")) \
    .withColumn("report_data", to_json(struct(*top_5_suppliers.columns))) \
    .select("report_type", "report_data")

df_avg_price_json = avg_price_by_supplier \
    .withColumn("report_type", lit("avg_product_price")) \
    .withColumn("report_data", to_json(struct(*avg_price_by_supplier.columns))) \
    .select("report_type", "report_data")

df_sales_country_json = sales_by_supplier_country \
    .withColumn("report_type", lit("sales_by_supplier_country")) \
    .withColumn("report_data", to_json(struct(*sales_by_supplier_country.columns))) \
    .select("report_type", "report_data")

supplier_report = df_top5_json.union(df_avg_price_json).union(df_sales_country_json)

In [42]:
supplier_report.write.mode("append").jdbc(url=ch_url, table="report_supplier", properties=ch_props)

### Качество продукции

In [ ]:
f_sales = spark.read.jdbc(url=pg_url, table="f_sales", properties=pg_props)
d_products = spark.read.jdbc(url=pg_url, table="d_products", properties=pg_props)

Продукты с наивысшим и наименьшим рейтингом.

In [44]:
from pyspark.sql.functions import sum

top_rating_products = (
    d_products.select("id", "name", "rating").orderBy("rating", ascending=False).limit(5)
)

lowest_rating_products = (
    d_products.select("id", "name", "rating").orderBy("rating", ascending=True).limit(5)
)

top_rating_products.show()
lowest_rating_products.show()

+----+---------+------+
|  id|     name|rating|
+----+---------+------+
| 586|  Cat Toy| 5.000|
| 985|Bird Cage| 5.000|
| 706|  Cat Toy| 5.000|
|8023|  Cat Toy| 5.000|
| 912| Dog Food| 5.000|
+----+---------+------+

+---+---------+------+
| id|     name|rating|
+---+---------+------+
|171|Bird Cage| 1.000|
|550| Dog Food| 1.000|
|233| Dog Food| 1.000|
|376| Dog Food| 1.000|
| 35|  Cat Toy| 1.000|
+---+---------+------+



Корреляция между рейтингом и объемом продаж.

In [54]:
from pyspark.sql.functions import sum
from pyspark.sql import Row

product_sales = (
    f_sales.groupBy("product_id")
           .agg(sum("quantity").alias("total_quantity"))
)

rating_vs_sales = (
    product_sales.join(d_products, product_sales.product_id == d_products.id)
                 .select("rating", "total_quantity")
)

correlation = rating_vs_sales.stat.corr("rating", "total_quantity")
correlation_df = spark.createDataFrame(
    [Row(metric="rating_vs_sales_correlation", value=correlation)]
)

correlation_df.show()

+--------------------+--------------------+
|              metric|               value|
+--------------------+--------------------+
|rating_vs_sales_c...|0.001004977801145763|
+--------------------+--------------------+



Продукты с наибольшим количеством отзывов.

In [56]:
most_reviewed_products = (
    d_products.select("id", "name", "reviews").orderBy("reviews", ascending=False).limit(5)
)

most_reviewed_products.show()

+----+---------+-------+
|  id|     name|reviews|
+----+---------+-------+
|4122|  Cat Toy|   1000|
|5256|Bird Cage|   1000|
|4224|Bird Cage|   1000|
|3505|  Cat Toy|   1000|
|4606|Bird Cage|   1000|
+----+---------+-------+



In [57]:
from pyspark.sql.functions import lit, to_json, struct

correlation_json = correlation_df \
    .withColumn("report_type", lit("rating_vs_sales_correlation")) \
    .withColumn("report_data", to_json(struct(*correlation_df.columns))) \
    .select("report_type", "report_data")

top_rated_json = top_rating_products \
    .withColumn("report_type", lit("top_rating_products")) \
    .withColumn("report_data", to_json(struct(*top_rating_products.columns))) \
    .select("report_type", "report_data")

low_rated_json = lowest_rating_products \
    .withColumn("report_type", lit("lowest_rating_products")) \
    .withColumn("report_data", to_json(struct(*lowest_rating_products.columns))) \
    .select("report_type", "report_data")

most_reviews_json = most_reviewed_products \
    .withColumn("report_type", lit("most_reviewed_products")) \
    .withColumn("report_data", to_json(struct(*most_reviewed_products.columns))) \
    .select("report_type", "report_data")

product_quality_report = (correlation_json.union(top_rated_json).union(low_rated_json).union(most_reviews_json))

In [58]:
product_quality_report.write.mode("append").jdbc(url=ch_url, table="report_quality", properties=ch_props)